<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/gpts/DIY_AK_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [36]:
#####################
# DATASET RETRIEVAL #
#####################

In [37]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-09-21 10:55:50--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-09-21 10:55:50 (17.5 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [38]:
with open("input.txt", mode="r") as f:
  text = f.read()

print(text[:10])

First Citi


In [39]:
###################
# HYPERPARAMETERS #
###################

In [207]:
block_size = 8
batch_size = 32
device = "cuda" if torch.cuda.is_available() else "cpu"
embed_dim = 32
n_epochs = 10
learning_rate = 1e-04
test_epochs = 200
eval_iters = 100

In [208]:
###############################
# TOKENIZATION AND DATALOADER #
###############################

In [209]:
vocab = list(sorted(set(text)))
text_to_num = {v:k for k,v in enumerate(vocab)}
num_to_text = {k:v for k,v in enumerate(vocab)}
vocab_size = len(vocab)

In [210]:
encode = lambda text: [text_to_num[t] for t in text]
decode = lambda numbers: "".join([num_to_text[n] for n in numbers])

In [211]:
decode(encode("Haubi"))

'Haubi'

In [212]:
data = torch.tensor(encode(text))
train_idx = int(0.9 * len(data))
train = data[:train_idx]
test = data[train_idx:]
len(train), len(test)

(1003854, 111540)

In [213]:
def get_batch(split):
  dataset = train if split == "train" else test
  idx = torch.randint(0, len(train) - block_size, (batch_size,))
  X = torch.stack([train[i:i +  block_size] for i in idx])
  Y = torch.stack([train[i + 1: i + block_size + 1] for i in idx])
  X, Y = X.to(device), Y.to(device)

  return X, Y

In [214]:
X, Y = get_batch("train")
X[1], Y[1]

(tensor([57, 53, 53, 52,  1, 51, 39, 49]),
 tensor([53, 53, 52,  1, 51, 39, 49, 43]))

In [234]:
@torch.no_grad()
def evaluate_model(model):
  out = {}
  model.eval()
  for split in ["train", "test"]:
    test_loss = torch.zeros(test_epochs)
    for epch in range(test_epochs):
      Xtst_b, Ytst_b = get_batch("test")
      _, loss = model(Xtst_b, Ytst_b)

      test_loss[epch] = loss.item()

    out[split] = test_loss.mean(dim=-1)

  model.train()

  return out

In [216]:
################
# BUILD MODELS #
################

In [237]:
class SimpleBiGramModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.emb = nn.Embedding(vocab_size, vocab_size)

  def forward(self, x, targets=None):
    logits = self.emb(x)
    if targets is not None:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
      return x, loss
    return x, None

  def generate(self, start_token, length):
    """
    start_token: (B, 1) tensor with the initial token(s)
    length: how many new tokens to generate
    """
    for _ in range(length):
      # only keep the last token
      input = start_token[:, -block_size:]   # shape (B, 1)

      # forward pass → logits for vocab
      logits = self(input)          # (B, 1, C)

      # take the logits at the last position
      print(logits)
      logits = logits[:, -1, :]     # (B, C)

      # turn into probabilities
      probs = F.softmax(logits, dim=-1)  # (B, C)

      # sample next token for each batch
      next_token = torch.multinomial(probs, num_samples=1)  # (B, 1)

      # append to sequence
      start_token = torch.cat((start_token, next_token), dim=1)

    return start_token


In [238]:
model = SimpleBiGramModel()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [239]:
for epoch in range(1000):

  if epoch % eval_iters == 0 or epoch == n_epochs - 1:
    out = evaluate_model(model)
    print(f"Epoch: {epoch}, train loss: {out["train"]:.4f}, test loss: {out["test"]:.4f}")

  X_batch, Y_batch = get_batch("train")
  logits, loss = model(X_batch, Y_batch)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  X_test_batch, Y_test_batch = get_batch("test")
  _, test_loss = model(X_test_batch, Y_test_batch)



Epoch: 0, train loss: 4.5324, test loss: 4.5365
Epoch: 9, train loss: 4.5455, test loss: 4.5298
Epoch: 100, train loss: 4.5277, test loss: 4.5099
Epoch: 200, train loss: 4.5083, test loss: 4.5124
Epoch: 300, train loss: 4.5013, test loss: 4.5118
Epoch: 400, train loss: 4.4914, test loss: 4.4942
Epoch: 500, train loss: 4.4730, test loss: 4.4808
Epoch: 600, train loss: 4.4668, test loss: 4.4583
Epoch: 700, train loss: 4.4592, test loss: 4.4527
Epoch: 800, train loss: 4.4480, test loss: 4.4429
Epoch: 900, train loss: 4.4330, test loss: 4.4288


In [240]:
generated_output = model.generate(torch.randint(0, 65, (1, 8)), 30)
output_list = generated_output.view(-1).tolist()
"".join([decode(output_list)])

(tensor([[51, 57, 36, 61,  9,  8, 20, 24]]), None)


TypeError: tuple indices must be integers or slices, not tuple